A diferencia de los modelos anteriores que consolidaba datos por período de consumos, en el presente script se busca pegar el consumo por PERIODO PRESTACION por persona y cada rango separado. 
Se eliminan cálculos importantes como la edad de la persona al momento del conusmo.

In [1]:
# Librerías usadas:
import pandas as pd
import numpy as np
import os
from datetime import datetime

In [2]:
# Parámetros de seteo.

# Parámetros de seteo.

# Ruta a la que tiene que salir la información
ruta_base = 'C:\\Users\\lsaucedo\\Desktop\\Brinkmann V1\\'

# Archivo de salida
archivo_salida = ruta_base + "PerPrest-Persona-Consumo-Bnknn_v2" + ".csv"

# Ruta en que se encuentran los archivos:
ruta = ruta_base + 'Consumo\\'


# 1- Levantar los archivos con consumos

In [3]:
# Levanto el consumo existente en una carpeta:

# Buscamos y levantamos a df todos los excel de "ruta"
contenido = os.listdir(ruta)

print("Inicio del proceso: " + datetime.now().strftime('%H:%M:%S'))

# Dataframe con el stock:
dfc = pd.DataFrame()

archivos = []
for fichero in contenido:
    if os.path.isfile(os.path.join(ruta, fichero)) and fichero.endswith('.xlsx'):
        df_temp = pd.read_excel(ruta+fichero)
        dfc = pd.concat([dfc, df_temp], axis=0)
        print("Se cargó el archivo: ", fichero, "a las: " + datetime.now().strftime('%H:%M:%S'))

df_temp = []
print("Fin de la carga de archivos")

Inicio del proceso: 13:25:47
Se cargó el archivo:  10.1 Consumos 202009-202109 (per prestac).xlsx a las: 13:27:11
Fin de la carga de archivos


# 1.2- Renombrar las columnas

In [4]:
# Se pueden renombrar incluso columnas que no existen (mejorar la eficiencia en función del warning)
dfc.rename( columns = {    'Periodo ID': 'Periodo',
                           'Persona ID': 'Persona',
                           'Persona Fecha Nacimiento': 'Fecha_nac',
                           'Discapacitado ID': 'Discapacidad',
                           'Marca Persona con Diabetes DESC': 'Diabetes',
                           'Marca Persona con Hepatitis DESC': 'Hepatitis',
                           'Marca Persona con PMI DESC': 'PMI',
                           'Marca Persona con Tratamiento Cronico DESC': 'Cronico',
                           'Marca Persona con Tratamiento Oncológico DESC':  'Oncologico',
                           'Sexo ID': 'Sexo',
                           'Plan Sin Segmento ID': 'Plan',
                           'Posicion Asociado DESC': 'Posicion Asociado',
                           'Zona de Promocion DESC': 'Zona de Promocion',
                           'Segmento Agrupado DESC': 'Segmento',
                           'Plan Agrupado Sin Segmento DESC': 'Plan Agrupado',
                           'Stock Asociados Salud': 'Stock',
                           'Edad Periodo ID': 'Edad',
                           'ACE Stock Asociados Salud': 'ACE', 
                           'Consumos Costo con Subsidios SSS': 'Consumo3',
                           'Recaudacion Costo con Otros Subsidios': 'Recaudacion3',
                           'Precio Sugerido Medicamento': "Precio_sug",
                           'Periodo Prestacion ID':'Periodo_Prest'
                           }, inplace=True)

In [5]:
dfc.columns

Index(['Periodo', 'Periodo_Prest', 'Persona', 'Fecha_nac', 'Edad Orden ID',
       'Sexo', 'Discapacidad', 'Posicion Asociado ID', 'Zona de Promocion ID',
       'Plan', 'Plan Agrupado', 'Origen Prestacion ID',
       'Origen Facturacion ID', 'Prestacion ID', 'Prestacion DESC',
       'Rubro Indicadores Consumo ID', 'Prestacion Acreedor ID',
       'Prestacion Acreedor DESC', 'Nomenclador ID', 'Nomenclador DESC',
       'Acreedor ID', 'Provision Acreedor ID', 'Posicion Acreedor ID',
       'Tipo Orden ID', 'Nro Orden ID', 'Tipo Orden Relacionada ID',
       'Orden Relacionada ID', 'Porcentaje Cobertura Medicamento ID',
       'Tipo de Venta DESC', 'Tipo de Venta ID', 'Subtipo de Venta DESC',
       'Subtipo de Venta ID', 'Segmento', 'Cantidad', 'Consumo', 'Precio_sug'],
      dtype='object')

# 2- Cálculo de edad

No se realiza el cálculo a partir de la fecha de nacimiento y período. Par eso ver versines 0.x

# 2- Agrego archivos auxiliares

No se agregan los diagnósticos

# 3- Agrego Marcas de Consumo y Rango Consumos

No se agregan rangos de consumos en esta versión. Bucar en las v0.x.

# Inspecciones varias

In [6]:
dfc['Nomenclador ID'].unique()

array(['NP', 'ME', 'NM', 'ND', nan, 'MD', 'M1', 'DI', 'NO', 'NU', 'NS',
       'OP', 'NB', 'NC', 'HB'], dtype=object)

In [7]:
dfc['Origen Prestacion ID'].unique()

array(['I', 'A', 'X'], dtype=object)

In [8]:
dfc['Origen Facturacion ID'].unique()

array(['F', 'A', 'R', 'C'], dtype=object)

# Segmentaciones y métricas

In [10]:
# Segmentaciones y métricas comúnes:

# QUEDA PENDIENTE SEGMENTO EN LA APERTURA
segmentacion = (['Periodo_Prest', 'Persona'])

# dejo para probar procesamientos más rápido (debugging)
segmentacion2 = ['Periodo', 'Sexo']


# Defino las métricas a calcular
metricas = ({'Consumo':[np.sum], 'Prestacion ID':pd.Series.nunique,'Cantidad':[np.sum]})


metricas2 = ({  'Consumo':[np.sum,len],
                'Cantidad':np.sum,
            })


# TD1 - Armado de la primera tabla de resúmen:
## Período PRESTACION - Persona
## A1 = Total
Se calcula para diferentes aperturas.

In [11]:
# Genero la tabla de salida
td1 = pd.pivot_table(dfc, index = segmentacion, 
               #values = ['Stock Asociados Salud', 'Edad Periodo ID'] , # no es obligatorio
               aggfunc = metricas)

In [12]:
# Renombramos columnas:

# Reseteo los multi index:
# td0 = td0.reset_index()


# Nombres de columnas:
col_td1 = [

    "Cantidad Total",
    "Consumo Total",
    "Prest. Dist. Total"  ]


# Renombro las columnas:
td1.columns = col_td1

# TD2 - Armado de aperturas diferentes al total
## A2: Por nomenclador y Origen Prestación

In [13]:
# Apertura para consumo agrupacion 2 (la 1 es la total)

condiciones2 = [
    (dfc['Origen Facturacion ID'] == 'A') ,
    (dfc['Origen Facturacion ID'] == 'C') ,
    (dfc['Nomenclador ID'] == 'ME') ,
    (dfc['Nomenclador ID'] == 'M1') ,
    (dfc['Origen Prestacion ID'] == 'I')]

# Lista de resultados en función de las selecciones
valores_condic2 = ['Naj', 'NC', 'MEd', 'MEd', 'Internacion sin A,C,Med']     

# Generación de la columna calculada
dfc['Agrc2'] = np.select(condiciones2, valores_condic2, default = 'Ambulatorio sin A,C,Med')

In [14]:
# No debería haber nan
dfc['Agrc2'].unique()

array(['Internacion sin A,C,Med', 'Ambulatorio sin A,C,Med', 'MEd', 'Naj',
       'NC'], dtype=object)

In [15]:
td2 = pd.pivot_table(dfc, index = segmentacion, columns = 'Agrc2',
               #values = ['Stock Asociados Salud', 'Edad Periodo ID'] , # no es obligatorio
               aggfunc = metricas, fill_value = 0)

In [16]:
# Nombres de columnas:
col_td2 = [
    
    "Cantidad Amb.",
    "Cantidad Int.",
    "Cantidad Medic.",
    "Cantidad NC.",
    "Cantidad NA",
    
    "Consumo Amb.",
    "Consumo Int.",
    "Consumo Medic.",
    "Consumo NC.",
    "Consumo NA",
    
    "Prest. Dist. Amb.",
    "Prest. Dist. Int.",
    "Prest. Dist. Medic.",
    "Prest. Dist. NC.",
    "Prest. Dist. NA"]

# Renombro las columnas:
td2.columns = col_td2


# TD2 - Amb por Rubros

## A3

In [17]:
# Apertura para consumo: Agrupador consumos 3

condiciones3 = [
    (dfc['Origen Prestacion ID'] == 'A') & (dfc['Rubro Indicadores Consumo ID'] == 'A'),
    
    (dfc['Origen Prestacion ID'] == 'A') & (dfc['Rubro Indicadores Consumo ID'] == 'I'),
    
    (dfc['Origen Prestacion ID'] == 'A') & (dfc['Rubro Indicadores Consumo ID'] == 'C'),
    (dfc['Origen Prestacion ID'] == 'A') & (dfc['Rubro Indicadores Consumo ID'] == 'D'),
    (dfc['Origen Prestacion ID'] == 'A') & (dfc['Rubro Indicadores Consumo ID'] == 'E'),
    (dfc['Origen Prestacion ID'] == 'A') & (dfc['Rubro Indicadores Consumo ID'] == 'F'),
    (dfc['Origen Prestacion ID'] == 'A') & (dfc['Rubro Indicadores Consumo ID'] == 'G'),
    
    (dfc['Origen Prestacion ID'] == 'A') & (dfc['Rubro Indicadores Consumo ID'] == 'N'),
    
    (dfc['Origen Prestacion ID'] == 'A') & (dfc['Rubro Indicadores Consumo ID'] == 'M'),
    
    (dfc['Origen Prestacion ID'] == 'A') & (dfc['Rubro Indicadores Consumo ID'] == 'J'),
    (dfc['Origen Prestacion ID'] == 'A') & (dfc['Rubro Indicadores Consumo ID'] == 'R'),
    (dfc['Origen Prestacion ID'] == 'A') & (dfc['Rubro Indicadores Consumo ID'] == 'S'),
    (dfc['Origen Prestacion ID'] == 'A') & (dfc['Rubro Indicadores Consumo ID'] == 'T'),
    (dfc['Origen Prestacion ID'] == 'A') & (dfc['Rubro Indicadores Consumo ID'] == 'U'),
    (dfc['Origen Prestacion ID'] == 'A') & (dfc['Rubro Indicadores Consumo ID'] == 'V'),
    (dfc['Origen Prestacion ID'] == 'A') & (dfc['Rubro Indicadores Consumo ID'] == 'W'),
    (dfc['Origen Prestacion ID'] == 'A') & (dfc['Rubro Indicadores Consumo ID'] == 'X'),
    
    (dfc['Origen Prestacion ID'] == 'A') ]

# Lista de resultados en función de las selecciones
valores_condic3 = ['Consulta', 'Laboratorio', 'Imagenes', 'Imagenes', 'Imagenes', 
                  'Imagenes', 'Imagenes', 'Fisiokinesio', 'Salud Mental', 
                  'Odontologia', 'Odontologia','Odontologia','Odontologia','Odontologia',
                  'Odontologia','Odontologia','Odontologia','Resto Amb']     

# Generación de la columna calculada
dfc['Agrc3'] = np.select(condiciones3, valores_condic3, default = 'No Considerar')


In [18]:
# Armado de la td:

td3 = pd.pivot_table(dfc, index = segmentacion, columns = 'Agrc3',
               #values = ['Stock Asociados Salud', 'Edad Periodo ID'] , # no es obligatorio
               aggfunc = metricas, fill_value = 0)

# td3.head()

In [19]:
td3.columns

MultiIndex([(     'Cantidad',     'sum',      'Consulta'),
            (     'Cantidad',     'sum',  'Fisiokinesio'),
            (     'Cantidad',     'sum',      'Imagenes'),
            (     'Cantidad',     'sum',   'Laboratorio'),
            (     'Cantidad',     'sum', 'No Considerar'),
            (     'Cantidad',     'sum',   'Odontologia'),
            (     'Cantidad',     'sum',     'Resto Amb'),
            (     'Cantidad',     'sum',  'Salud Mental'),
            (      'Consumo',     'sum',      'Consulta'),
            (      'Consumo',     'sum',  'Fisiokinesio'),
            (      'Consumo',     'sum',      'Imagenes'),
            (      'Consumo',     'sum',   'Laboratorio'),
            (      'Consumo',     'sum', 'No Considerar'),
            (      'Consumo',     'sum',   'Odontologia'),
            (      'Consumo',     'sum',     'Resto Amb'),
            (      'Consumo',     'sum',  'Salud Mental'),
            ('Prestacion ID', 'nunique',      'Consulta'

In [20]:
# Nombres de columnas:
col_td3 = [
    
    "Cantidad Consulta",
    "Cantidad Fisiokinesio",
    "Cantidad Imagenes",
    "Cantidad Laboratorio",
    "Cantidad No Considerar",
    "Cantidad Odontologia",
    "Cantidad Resto Amb",
    "Cantidad Salud Mental",
    
    "Consumo Consulta",
    "Consumo Fisiokinesio",
    "Consumo Imagenes",    
    "Consumo Laboratorio",
    "Consumo No Considerar",
    "Consumo Odontologia",
    "Consumo Resto Amb",
    "Consumo Salud Mental",

    "Prest. Dist. Consulta",
    "Prest. Dist. Fisiokinesio",
    "Prest. Dist. Imagenes",    
    "Prest. Dist. Laboratorio",
    "Prest. Dist. No Considerar",
    "Prest. Dist. Odontologia",
    "Prest. Dist. Resto Amb",
    "Prest. Dist. Salud Mental"]

# Renombro las columnas:
td3.columns = col_td3

# TD4 - Medicamentos por Origen Prestación

## Agr 4: Medicamentos por origen

In [21]:
# Apertura para consumo: Agrupador consumos 3

condiciones4 = [
    
    # Condiciones para marcar medicamentos en internacion:
    (dfc['Origen Prestacion ID'] == 'I') & (dfc['Nomenclador ID'] == 'ME'),
    (dfc['Origen Prestacion ID'] == 'I') & (dfc['Nomenclador ID'] == 'M1'),
    
    # Condiciones para marcar medicamentos en amb -> no se filtra origen (por defecto mandar a AMB)
    (dfc['Nomenclador ID'] == 'ME'),
    (dfc['Nomenclador ID'] == 'M1') ]

# Lista de resultados en función de las selecciones
valores_condic4 = ['Med. Int', 'Med. Int', 'Med. Amb', 'Med. Amb']     

# Generación de la columna calculada
dfc['Agrc4'] = np.select(condiciones4, valores_condic4, default = 'No Considerar')


In [22]:
# Armado de la td:

td4 = pd.pivot_table(dfc, index = segmentacion, columns = 'Agrc4',
               #values = ['Stock Asociados Salud', 'Edad Periodo ID'] , # no es obligatorio
               aggfunc = metricas, fill_value = 0)

# td4.head()

In [23]:
td4.columns

MultiIndex([(     'Cantidad',     'sum',      'Med. Amb'),
            (     'Cantidad',     'sum',      'Med. Int'),
            (     'Cantidad',     'sum', 'No Considerar'),
            (      'Consumo',     'sum',      'Med. Amb'),
            (      'Consumo',     'sum',      'Med. Int'),
            (      'Consumo',     'sum', 'No Considerar'),
            ('Prestacion ID', 'nunique',      'Med. Amb'),
            ('Prestacion ID', 'nunique',      'Med. Int'),
            ('Prestacion ID', 'nunique', 'No Considerar')],
           names=[None, None, 'Agrc4'])

In [24]:
# Nombres de columnas:
col_td4 = [
    
    "Cantidad Med. Amb",
    "Cantidad Med. Int",
    "Cantidad No Considerar",
    
    "Consumo Med. Amb",
    "Consumo Med. Int",
    "Consumo No Considerar",    


    "Prest. Dist. Med. Amb",
    "Prest. Dist. Med. Int",
    "Prest. No Considerar"]

# Renombro las columnas:
td4.columns = col_td4

# TD5 - Medicamentos por vía

## Agr 5: Medicamentos por vía

In [25]:
# Apertura para consumo: Agrupador consumos 3

condiciones5 = [
    
    # Condiciones para marcar medicamentos en internacion:
    (dfc['Provision Acreedor ID'] == 6) & (dfc['Nomenclador ID'] == 'ME'),
    (dfc['Provision Acreedor ID'] == 6) & (dfc['Nomenclador ID'] == 'M1'),

    (dfc['Provision Acreedor ID'] == 66) & (dfc['Nomenclador ID'] == 'ME'),
    (dfc['Provision Acreedor ID'] == 66) & (dfc['Nomenclador ID'] == 'M1'),
    
    (dfc['Nomenclador ID'] == 'ME'),
    (dfc['Nomenclador ID'] == 'M1') ]

# Lista de resultados en función de las selecciones
valores_condic5 = ['Farmacia', 'Farmacia', 'Provision', 'Provision', 'Otras vias', 'Otras vias']     

# Generación de la columna calculada
dfc['Agrc5'] = np.select(condiciones5, valores_condic5, default = 'No Considerar')


In [26]:
# Armado de la td:

td5 = pd.pivot_table(dfc, index = segmentacion, columns = 'Agrc5',
               #values = ['Stock Asociados Salud', 'Edad Periodo ID'] , # no es obligatorio
               aggfunc = metricas, fill_value = 0)

# td5.head()

In [27]:
td5.columns

MultiIndex([(     'Cantidad',     'sum',      'Farmacia'),
            (     'Cantidad',     'sum', 'No Considerar'),
            (     'Cantidad',     'sum',    'Otras vias'),
            (     'Cantidad',     'sum',     'Provision'),
            (      'Consumo',     'sum',      'Farmacia'),
            (      'Consumo',     'sum', 'No Considerar'),
            (      'Consumo',     'sum',    'Otras vias'),
            (      'Consumo',     'sum',     'Provision'),
            ('Prestacion ID', 'nunique',      'Farmacia'),
            ('Prestacion ID', 'nunique', 'No Considerar'),
            ('Prestacion ID', 'nunique',    'Otras vias'),
            ('Prestacion ID', 'nunique',     'Provision')],
           names=[None, None, 'Agrc5'])

In [28]:
# Nombres de columnas:
col_td5 = [
    
    "Cantidad Farmacia",
    "Cantidad No Considerar",
    "Cantidad Otras vias",
    "Cantidad Provision",
    
    "Consumo Farmacia",
    "Consumo No Considerar",
    "Consumo Otras vias",    
    "Consumo Provision",  

    "Prest. Farmacia",
    "Prest. No Considerar",
    "Prest. Otras vias",
    "Prest. Provision"]

# Renombro las columnas:
td5.columns = col_td5

# Agrupamos las diferentes tablas

In [29]:
tdt = pd.concat([td1, td2, td3, td4, td5], axis=1)
tdt.reset_index()
tdt.head()

Cantidad Total  Consumo Total  Prest. Dist. Total  \
Periodo_Prest Persona                                                      
202009        266                   6         -46.33                   5   
              452                   4          61.46                   4   
              779                   8        2005.31                   7   
              780                  20        6672.98                   9   
              1219                  7        -704.21                   6   

                       Cantidad Amb.  Cantidad Int.  Cantidad Medic.  \
Periodo_Prest Persona                                                  
202009        266                  0              0                0   
              452                  0              0                0   
              779                  3              0                2   
              780                 14              0                0   
              1219                 0              0                0   

                       Cantidad NC.  Cantidad NA  Consumo Amb.  Consumo Int.  \
Periodo_Prest Persona                                                          
202009        266                 2            4          0.00           0.0   
              452                 3            1          0.00           0.0   
              779                 2            1       1376.54           0.0   
              780                 2            4       6709.87           0.0   
              1219                3            4          0.00           0.0   

                       ...  Cantidad Otras vias  Cantidad Provision  \
Periodo_Prest Persona  ...                                            
202009        266      ...                    0                   0   
              452      ...                    0                   0   
              779      ...                    0                   0   
              780      ...                    0                   0   
              1219     ...                    0                   0   

                       Consumo Farmacia  Consumo No Considerar  \
Periodo_Prest Persona                                            
202009        266                  0.00                 -46.33   
              452                  0.00                  61.46   
              779                563.27                1442.04   
              780                  0.00                6672.98   
              1219                 0.00                -704.21   

                       Consumo Otras vias  Consumo Provision  Prest. Farmacia  \
Periodo_Prest Persona                                                           
202009        266                     0.0                0.0                0   
              452                     0.0                0.0                0   
              779                     0.0                0.0                2   
              780                     0.0                0.0                0   
              1219                    0.0                0.0                0   

                       Prest. No Considerar  Prest. Otras vias  \
Periodo_Prest Persona                                            
202009        266                         5                  0   
              452                         4                  0   
              779                         5                  0   
              780                         9                  0   
              1219                        6                  0   

                       Prest. Provision  
Periodo_Prest Persona                    
202009        266                     0  
              452                     0  
              779                     0  
              780                     0  
              1219                    0  

[5 rows x 63 columns]

# 4- Exportar la información a un .csv


Dentro del código queda pendiente:
1- Calcular rango de edad.
2- Cargar el diganóstico de todo el stock para levantar una vez.
3- Definir una función para cambio de nombres.
4- Definir una función para carga de dataframes desde excel.
5- Definir todos los parámetros que fueron necesarios.



In [30]:
# Genero un dataframe con la td
base_consumos = pd.DataFrame(tdt)

# Exporto la información en csv:
base_consumos.to_csv(archivo_salida, encoding = 'latin1', decimal=',')
# A modo de ejemplo:
#  df.to_csv('./exportaciones/' + supermercado + '-' + str(fecha_exportado) + '.csv', encoding = 'latin1')